### Opening

In [23]:
import os
import pandas as pd

folder = "/Users/luisfaria/Desktop/sEngineer/masters_SWEAI/2025-T2/T2-REM/assignments/Assessment2/nps"

# If in codespaces:
# folder = "/workspaces/masters-swe-ai/2025-T2/T2-REM/assignments/Assessment2/nps"

# Open all .xlsx files in the folder
for file in os.listdir(folder):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(folder, file))
        print(df.shape)

(27373, 5)


### Treating

In [24]:
# Extract the month from the 'Date' column
df['Month'] = pd.to_datetime(df['Date']).dt.month

In [25]:
def categorize_nps(score):
    if score >= 9:
        return 'Promoter'
    elif score >= 7:
        return 'Passive'
    else:
        return 'Detractor'

df['Category'] = df['Score'].apply(categorize_nps)

### Checking

In [26]:
# Group the DataFrame by 'Store' and 'Month'
nps_by_store_month = df.groupby(['Store', 'Month'])['Category'].value_counts(normalize=True).unstack(fill_value=0)

# Calculate the NPS for each group
nps_by_store_month['NPS'] = (nps_by_store_month['Promoter'] - nps_by_store_month['Detractor']) * 100

In [27]:
nps_alphaville = df.loc[df['Store'] == 'Alphaville']
nps_alphaville_by_store_by_month = nps_alphaville.groupby(['Store', 'Month', 'Year'])['Category'].value_counts(normalize=True).unstack(fill_value=0)
nps_alphaville_by_store_by_month['NPS'] = (nps_alphaville_by_store_by_month['Promoter'] - nps_alphaville_by_store_by_month['Detractor']) * 100
nps_alphaville_by_store_by_month

Category               Detractor   Passive  Promoter         NPS
Store      Month Year                                           
Alphaville 1     2025   0.000000  0.000000  1.000000  100.000000
           2     2025   0.000000  0.000000  1.000000  100.000000
           5     2024   0.000000  0.015625  0.984375   98.437500
           6     2024   0.012048  0.036145  0.951807   93.975904
           7     2024   0.000000  0.028777  0.971223   97.122302
           8     2024   0.023810  0.023810  0.952381   92.857143
           9     2024   0.000000  0.037037  0.962963   96.296296
           10    2024   0.000000  0.000000  1.000000  100.000000
           11    2024   0.000000  0.000000  1.000000  100.000000
           12    2024   1.000000  0.000000  0.000000 -100.000000

### Preparing to merge

In [28]:
# I need to:
# 1 - Create a new column called "month" that will have YYYY-MM
# 2 - Create a new dataframe called "df_nps_store_monthly" with the following columns: month, Store, NPS (needs to be calculated with the "Category" info), count_of_responses

In [30]:
df.columns

Index(['Date', 'Score', 'Store', 'Year', 'Comment', 'Month', 'Category'], dtype='object')

In [31]:
# Assuming the dataframe is called 'df'
# Step 1: Create 'month' column by merging 'Year' and 'Month'
df['month'] = df['Year'].astype(str) + '-' + df['Month'].astype(str).str.zfill(2)

# Step 2: Create df_nps_store_monthly
df_nps_store_monthly = df.groupby(['month', 'Store']).agg(
    count_of_responses=('Score', 'count'),
    promoters=('Category', lambda x: (x == 'Promoter').sum()),
    detractors=('Category', lambda x: (x == 'Detractor').sum())
).reset_index()

# Calculate NPS: (% Promoters - % Detractors) * 100
df_nps_store_monthly['NPS'] = (
    (df_nps_store_monthly['promoters'] / df_nps_store_monthly['count_of_responses'] -
     df_nps_store_monthly['detractors'] / df_nps_store_monthly['count_of_responses']) * 100
)

# Drop temporary columns if needed
df_nps_store_monthly = df_nps_store_monthly[['month', 'Store', 'NPS', 'count_of_responses']]

In [34]:
df_nps_store_monthly

,month,Store,NPS,count_of_responses
0,2022-02,Barra da Tijuca,100.000000,30
1,2022-02,Belo Horizonte,85.714286,49
2,2022-02,Campinas,75.925926,54
3,2022-02,Copacabana,77.464789,71
4,2022-02,Ipiranga,81.578947,76
...,...,...,...,...
649,2025-05,Jardins,54.545455,11
650,2025-05,Mooca,0.000000,2
651,2025-05,São Bernardo,100.000000,10
652,2025-05,Tatuapé,80.000000,10
